## Carregar os módulos necessários

In [256]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium.plugins import MarkerCluster

print(np)
print(pd)
print(sns)
print(plt)
print(folium)
print(MarkerCluster)

<module 'numpy' from '/home/chico_linux/.local/lib/python3.10/site-packages/numpy/__init__.py'>
<module 'pandas' from '/home/chico_linux/.local/lib/python3.10/site-packages/pandas/__init__.py'>
<module 'seaborn' from '/home/chico_linux/.local/lib/python3.10/site-packages/seaborn/__init__.py'>
<module 'matplotlib.pyplot' from '/home/chico_linux/.local/lib/python3.10/site-packages/matplotlib/pyplot.py'>
<module 'folium' from '/home/chico_linux/.local/lib/python3.10/site-packages/folium/__init__.py'>
<class 'folium.plugins.marker_cluster.MarkerCluster'>


In [257]:
marine_df = pd.read_csv('src/data/terreno_marinha.csv', 
                        index_col=False,
                        header=0,
                        names=['class', 'rip_id', 'rip_use', 'register_date', 'state', 'city', 'address', 'neighborhood', 'lat', 'long', 'precision_level', 'concept', 'type', 'status', 'owner', 'use_date', 'area', 'owner_area'],
                        sep= ';', 
                        on_bad_lines = 'skip'
                        )

marine_df['register_date'] = pd.to_datetime(marine_df['register_date'], format='%d/%m/%Y', errors='coerce')
marine_df['use_date'] = pd.to_datetime(marine_df['use_date'], format='%d/%m/%Y', errors='coerce')

marine_df = marine_df[(marine_df['lat'] != '-')]
marine_df['lat'] = marine_df['lat'].astype(float)
marine_df['long'] = marine_df['long'].astype(float)

marine_df['area'] = marine_df['area'].str.replace(',', '').astype(float)
marine_df['owner_area'] = marine_df['owner_area'].str.replace(',', '').astype(float)

marine_df

,class,rip_id,rip_use,register_date,state,city,address,neighborhood,lat,long,precision_level,concept,type,status,owner,use_date,area,owner_area
0,Uso Especial,1001905005,1001915000,2008-11-07,RO,Guajara-Mirim,AV Beira Rio s/n Lote da Marinha - Quadra 94-...,Centro,-10.792542,-65.347723,Manual - Na rua ou na via de acesso rural,Marinha com Nacional Interior,Porto,Em Processso de Destinação,União (Adm. Pub. Fed. direta),2008-11-07,3550.00,3550.00
1,Uso Especial,3006855002,3006865008,2012-07-02,RO,Porto Velho,AV Rogério Weber s/nº Setor 03 - Parque das Águas,Baixa da União,-8.766495,-63.906223,Geocodificado - Endereço completo encontrado,Marinha com Nacional Interior,Parque,Em Processso de Destinação,União (Adm. Pub. Fed. direta),2010-06-01,211878.58,211878.58
2,Uso Especial,3007405000,3007415006,2016-05-19,RO,Porto Velho,"EST do Terminal 400 Setor 08, Quadra 501, Lote...",Panair,-8.745989,-63.916227,Manual - Na porta ou na sede do imóvel rural,Marinha com Nacional Interior,Terreno,Em Processso de Destinação,União (Adm. Pub. Fed. direta),2002-02-15,173856.31,173856.31
3,Uso Especial,3007975001,3007985007,2018-09-27,RO,Porto Velho,"MARG Esquerda do Rio Madeira, ao lado da BR 31...",Zona Rural,-8.737915,-63.925847,Sistemas da SPU,Marinha com Acrescido,Terreno,Em Processso de Destinação,União (Adm. Pub. Fed. direta),2019-11-01,8568.22,8568.22
4,Uso Especial,3008015001,3008025007,2019-01-07,RO,Porto Velho,AV Governador Jorge Teixeira 3146 Instituto Fe...,Setor Indústrial,-8.741097,-63.889087,Geocodificado - Endereço completo encontrado,Marinha com Nacional Interior,Escola,Uso próprio em serviço público,Fundação ou Autarquia (Adm. Pub. Fed. indireta),2015-12-07,15000.00,15000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379220,Dominial,9985010005752,1,2020-05-25,SC,Itapoa,AV BEIRA MAR V S/N,FIGUEIRA DO PONTAL,-26.180952,-48.601293,Manual - Na rua ou na via de acesso rural,Marinha,Dominial,Inscrição de Ocupação,União (Adm. Pub. Fed. direta),NaT,342.07,342.07
379221,Dominial,9985010006058,1,2021-02-03,SC,Itapoa,R DAS PALMEIRAS 34 RUA - 1630,PRINCESA DO MAR,-26.135650,-48.608624,Manual - Na rua ou na via de acesso rural,Marinha,Dominial,Inscrição de Ocupação,União (Adm. Pub. Fed. direta),NaT,768.00,747.84
379222,Dominial,9985010006139,3,1996-11-07,SC,Itapoa,AV BEIRA MAR 2068 LOTE 266A,ITAPEMA DO NORTE,-26.086830,-48.603718,Manual - Na rua ou na via de acesso rural,Marinha,Dominial,Inscrição de Ocupação,União (Adm. Pub. Fed. direta),NaT,373.37,373.37
379223,Dominial,9985010006210,2,1996-11-07,SC,Itapoa,R PEDRA DA MEDANHA 41 LOTE 266B,ITAPEMA DO NORTE,-26.069291,-48.618379,Manual - No Bairro ou na localidade rural,Marinha,Dominial,Inscrição de Ocupação,União (Adm. Pub. Fed. direta),NaT,375.00,375.00


In [258]:
marine_df.groupby(['type'], as_index=False).agg({'rip_id': 'count'}).sort_values(['rip_id'], ascending=False)

,type,rip_id
15,Dominial,358594
50,Terreno,590
24,Farol (farolete),244
18,Espelho D'Água,152
31,Ilha (ilhota),118
16,Edifício / Prédio,98
3,Apartamento,93
36,Outros,89
9,Casa,87
51,Universidade,67


In [259]:
# marine_df = marine_df[marine_df.type.isin(['Terreno', 'Gleba', 'Reserva', 'Ilha (ilhota)', "Espelho D'Água"])]
marine_df = marine_df[marine_df.type.isin(['Terreno', 'Gleba', 'Reserva', 'Ilha (ilhota)'])]

In [260]:
marine_df = marine_df[~(marine_df.state.isin(['DF', 'MT', 'MG', 'AC', "RN", 'RO', 'AM']))]

In [261]:
marine_df.groupby(['concept'], as_index=False).agg({'rip_id': 'count'}).sort_values(['rip_id'], ascending=False)

,concept,rip_id
0,Marinha,353
1,Marinha com Acrescido,219
2,Marinha com Nacional Interior,142


In [262]:
# marine_df['tooltip'] = marine_df['type'] + ' - ' + str(marine_df['rip_id']).split('\n')[0]
# marine_df['tooltip'] = marine_df['type'] + ' - ' + str(marine_df['rip_use']).split('\n')[0]
marine_df['tooltip'] = marine_df['type'] + ' - ' + marine_df['rip_id'].astype(str)

In [263]:
marine_df['color'] = np.where(marine_df['type'] == 'Terreno', '#641E16', '#154360')
marine_df['color'] = np.where(marine_df['type'] == 'Gleba', '#5DADE2', marine_df['color'])
marine_df['color'] = np.where(marine_df['type'] == 'Reserva', '#58D68D', marine_df['color'])
marine_df['color'] = np.where(marine_df['type'] == 'Ilha (ilhota)', '#E74C3C', marine_df['color'])

In [264]:
marine_df = marine_df.drop_duplicates(subset=['area', 'lat', 'long'])

In [265]:
marine_df = marine_df[~(marine_df.lat == -5.092011) & ~(marine_df.long == -42.80376)]
marine_df = marine_df[~(marine_df.lat == -5.366336945) & ~(marine_df.long == -49.12535756)]
marine_df = marine_df[~(marine_df.lat == -5.349939449) & ~(marine_df.long == -49.13003707)]
marine_df = marine_df[~(marine_df.lat == -28.62924658) & ~(marine_df.long == -56.0263081)]
marine_df = marine_df[~(marine_df.lat == -27.5723604) & ~(marine_df.long == -54.6710219)]
marine_df = marine_df[~(marine_df.lat == -22.774078) & ~(marine_df.long == -53.303135)]
marine_df = marine_df[~(marine_df.lat == -22.41771102) & ~(marine_df.long == -50.57502853)]
marine_df = marine_df[~(marine_df.lat == -22.59499466) & ~(marine_df.long == -48.79830363)]
marine_df = marine_df[~(marine_df.lat == -22.498636) & ~(marine_df.long == -48.557842)]
marine_df = marine_df[~(marine_df.lat == -22.49134) & ~(marine_df.long == -48.558807)]
marine_df = marine_df[~(marine_df.lat == -22.81175045) & ~(marine_df.long == -45.18986504)]
marine_df = marine_df[~(marine_df.lat == -9.405954711) & ~(marine_df.long == -40.50411564)]
marine_df = marine_df[~(marine_df.lat == -8.319732711) & ~(marine_df.long == -38.74383868)]
marine_df = marine_df[~(marine_df.lat == -5.097750713) & ~(marine_df.long == -38.37050173)]

In [266]:
my_map = folium.Map(
                    location = [-11.733350280370129, -49.05221944693883], # Centralizar no Brasil
                    zoom_start = 5,
                    tiles = 'OpenStreetMap'
                    )

lat = marine_df[marine_df.lat.notnull()].lat.tolist()
long = marine_df[marine_df.long.notnull()].long.tolist()
marine_info = marine_df.tooltip.tolist()
marine_color = marine_df.color.tolist()

for x, y, z, w in zip(lat, long, marine_info, marine_color):

    folium.CircleMarker(location=[x, y], 
                        fill=True, 
                        tooltip=z,
                        # radius=s,
                        color=w).add_to(my_map)

In [267]:
my_map

In [268]:
# attr = ('&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')
# attr = ('&copy; <a href="https://www.stadiamaps.com/" target="_blank">Stadia Maps</a> &copy; <a href="https://www.stamen.com/" target="_blank">Stamen Design</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

# tiles = "https://tile.openstreetmap.org/{z}/{x}/{y}.png"
# tiles = "https://tiles.stadiamaps.com/tiles/stamen_toner/{z}/{x}/{y}{r}.png"

my_map = folium.Map(
                    location = [-11.733350280370129, -49.05221944693883], # Centralizar no Brasil
                    zoom_start = 5,
                    tiles = 'Cartodb Positron'
                    # tiles = tiles,
                    # attr=attr
                    )

locations = list(zip(lat, long))

MarkerCluster(locations=locations, 
                    popups=locations,
                    ).add_to(my_map)

In [269]:
my_map

In [270]:
# marine_df[(marine_df.lat == -23.96304749) & (marine_df.long == -46.3047431)]
# marine_df[(marine_df.lat == -22.78824135) & (marine_df.long == -43.21507248)].count()

# marine_df[(marine_df.neighborhood.str.contains('ESTUARIO'))].rip_id.count()